In [1]:
# ==============================================================

# SIMPLE CHAT APP - Google Colab Ready

# ==============================================================

# Copy-paste this entire file into a Colab cell!

#

# This example shows TWO ways to build nodes:

# 1. @agora_node decorator (NEW! simpler!)

# 2. TracedAsyncNode classes (classic approach)

# ==============================================================



# Install dependencies

!pip install -q openai traceloop-sdk nest_asyncio
!pip install --force-reinstall -q git+https://github.com/thebnbrkr/Agora-test.git



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.5/357.5 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.17.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.17.0 require

In [11]:

import nest_asyncio
import os
import asyncio
import time
from openai import OpenAI

nest_asyncio.apply()

from agora.agora_tracer import (
    TracedAsyncNode,
    TracedAsyncFlow,
    init_traceloop,
    agora_node,  # ⭐ NEW DECORATOR!

)




In [12]:

# ==============================================================
# SETUP
# ==============================================================

# Initialize Traceloop

LOG_DIR = "/content/chat_telemetry"
os.makedirs(LOG_DIR, exist_ok=True)

init_traceloop(
    app_name="simple_chat",
    export_to_console=True,
    export_to_file=f"{LOG_DIR}/traces_{int(time.time())}.jsonl",
)

⚠️  Traceloop already initialized


In [14]:

# Setup OpenAI
os.environ["OPENAI_API_KEY"] = ""  # ⚠️ Replace with your key!
client = OpenAI()

# ==============================================================
# METHOD 1: Using @agora_node Decorator (SIMPLER!)
# ==============================================================
# The decorator wraps your function into a TracedAsyncNode.
# Your function receives the shared dict and returns the next action.
# ==============================================================





@agora_node(name="GetUserInput")
async def get_user_input(shared):
    """Get input from user - decorated function approach"""
    # Initialize chat on first turn

    if "messages" not in shared:
        shared["messages"] = []
        shared["turn"] = 0
        print("=" * 60)
        print("💬 CHAT READY - Type 'exit' to quit")
        print("=" * 60)
        print()
    # Get user input

    user_input = await asyncio.to_thread(input, "👤 You: ")
    user_input = user_input.strip()
    # Handle exit

    if user_input.lower() in ["exit", "quit", "bye"]:
        return "exit"

    # Store message
    shared["turn"] += 1
    shared["messages"].append({"role": "user", "content": user_input})

    # Return next action for routing
    return "respond"


@agora_node(name="GenerateResponse")
async def generate_response(shared):
    """Generate AI response - decorated function approach"""

    messages = shared["messages"]
    start = time.time()

    # Call OpenAI (Traceloop auto-instruments this!)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7,
        max_tokens=150,
    )


    content = response.choices[0].message.content.strip()
    print(f"\n🤖 Bot: {content}\n")  # Just show the message
    shared["messages"].append({"role": "assistant", "content": content})


    # latency = (time.time() - start) * 1000
    # content = response.choices[0].message.content.strip()
    # tokens = response.usage.total_tokens



    # Print response

    # print(f"\n🤖 Bot: {content}")
    # print(f"⏱️  {latency:.0f}ms | {tokens} tokens\n")
    # Store assistant message
    # shared["messages"].append({"role": "assistant", "content": content})

    # Return next action

    return "input"




In [15]:



# ==============================================================
# METHOD 2: Using TracedAsyncNode Classes (Classic)
# ==============================================================
# This approach gives you more control with prep/exec/post phases.
# ==============================================================


class ExitNode(TracedAsyncNode):
    """Exit the chat gracefully"""
    async def exec_async(self, prep_res):
        print()
        print("=" * 60)
        print("👋 Thanks for chatting! Goodbye!")
        print("=" * 60)
        return None



In [16]:

# ==============================================================
# MAIN CHAT FLOW
# ==============================================================

async def run_chat():
    """Main chat flow with routing"""

    # Create nodes using BOTH approaches:
    # - Decorated functions (get_user_input, generate_response)
    # - Class instance (exit_node)

    input_node = get_user_input  # ⭐ This is already a TracedAsyncNode!
    response_node = generate_response  # ⭐ This too!
    exit_node = ExitNode("ChatExit")

    # Build flow with transitions
    flow = TracedAsyncFlow("ChatFlow")
    flow.start(input_node)

    # Define routing
    input_node - "respond" >> response_node
    input_node - "exit" >> exit_node
    response_node - "input" >> input_node

    # Run the flow

    shared = {}
    await flow.run_async(shared)
    # Print summary
    print()
    print("=" * 60)
    print("📊 SESSION SUMMARY")
    print("=" * 60)
    print(f"Total turns: {shared.get('turn', 0)}")
    print(f"Total messages: {len(shared.get('messages', []))}")
    print()
    print("🔍 Flow structure:")
    print(flow.to_mermaid())
    print("=" * 60)
    print()
    print(f"📁 Traces saved to: {LOG_DIR}")
    print("=" * 60)

In [17]:


# ==============================================================
# RUN IT!
# ==============================================================

# Run the chat
await run_chat()

# ==============================================================
# KEY FEATURES:
# ==============================================================
# ✅ @agora_node decorator - wrap any function into a node!
# ✅ Automatic OpenTelemetry tracing via Traceloop
# ✅ State management via shared dict
# ✅ Routing based on return values
# ✅ Mix decorators and classes freely
# ✅ Full telemetry export (console + file)

{
    "name": "GetUserInput.prep",
    "context": {
        "trace_id": "0xed6bb32cbc51a3c81617d4c7442c3f1f",
        "span_id": "0xb45b24bc4661a110",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x6a9788f8c8db10d4",
    "start_time": "2025-11-10T10:06:53.394482Z",
    "end_time": "2025-11-10T10:06:53.394515Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "agora.node": "GetUserInput",
        "agora.phase": "prep",
        "duration_ms": 0.0
    },
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "telemetry.sdk.language": "python",
            "telemetry.sdk.name": "opentelemetry",
            "telemetry.sdk.version": "1.38.0",
            "service.name": "simple_chat"
        },
        "schema_url": ""
    }
}
💬 CHAT READY - Type 'exit' to quit

👤 You: howdy
{
    "name": "GetUserInput.exec",
    "context": {
        "trace_id": "0xed6bb32cbc51a3c81617d4c7442c3f1f",
 